In [ ]:
import yaml
import pickle
import os.path as osp
import subprocess
from urllib.parse import urlparse
import numpy as np
import matplotlib.pyplot as plt
from utils import time_intp, str2time, filter_nan_values, read_pkl, read_yml, retrieve_url

## Setup

In [ ]:
retrieve_url("https://demo.openwfm.org/web/data/fmda/dicts/fmda_nw_202401-05_f05.pkl", "data/fmda_nw_202401-05_f05.pkl")

In [ ]:
data_params = read_yml("params_data.yaml")
data_params

In [ ]:
# dat = read_pkl("data/test_CA_202401.pkl")
dat = read_pkl("data/test_NW_202401.pkl")

## Filters

In [ ]:
# Useful Cases:
    # NV040_202401: more raws observations than HRRR, interp should shorten
    # NV026_202401: raws 10min obs, interp should shorten
    # CGVC1_202401: missing only a few observations, interp should lengthen
    # YNWC1_202401: only 2 observations, should be filtered entirely

In [ ]:
def flag_lag_stretches(x, lag = 1, threshold = data_params['zero_lag_threshold']):
    lags = np.diff(x, n=lag)
    zero_lag_indices = np.where(lags == 0)[0]
    current_run_length = 1
    for i in range(1, len(zero_lag_indices)):
        if zero_lag_indices[i] == zero_lag_indices[i-1] + 1:
            current_run_length += 1
            if current_run_length > threshold:
                return True
        else:
            current_run_length = 1
    else:
        return False    

In [ ]:
cases = list([*dat.keys()])
flags = np.zeros(len(cases))
for i, case in enumerate(cases):
    print("~"*50)
    print(f"Case: {case}")
    time_raws=str2time(dat[case]['RAWS']['time_raws'])
    time_hrrr=str2time(dat[case]["HRRR"]['time'])
    fm = dat[case]['RAWS']['fm']
    ynew = time_intp(time_raws,fm,time_hrrr)
    dat[case]['y'] = ynew
    if flag_lag_stretches(ynew):
        print(f"Flagging case {case} for zero lag stretches greater than `zero_lag_threshold` param {data_params['zero_lag_threshold']}")
        flags[i]=1
    if flag_lag_stretches(ynew, lag=2):
        print(f"Flagging case {case} for constant linear stretches greater than `max_intp_time` param {data_params['max_intp_time']}")
        flags[i]=1
    if np.any(ynew>=data_params['max_fm']) or np.any(ynew<=data_params['min_fm']):
        print(f"Flagging case {case} for FMC outside param range {data_params['min_fm'],data_params['max_fm']}. FMC range for {case}: {ynew.min(),ynew.max()}")
        flags[i]=1

In [ ]:
flagged_cases = [element for element, flag in zip(cases, flags) if flag == 1]
print(flagged_cases)